In [17]:
import pandas as pd
import numpy as np
from numba import jit
import re

In [2]:
# define the file path/names
fname_3a4='input_data/FOC 3A4.csv'
fname_supply='input_data/PCBA&DF SUPPLY.xlsx'
fname_ct2r='input_data/CPN CT2R.xlsx'
f_df_supply=''
f_cap=''
f_flt=''
fname_bom='input_data/PABU BOM.xlsx'

In [3]:
# read files into df
df_3a4=pd.read_csv(fname_3a4,encoding='iso-8859-1',low_memory=False)#parse_dates=['CURRENT_FCD_NBD_DATE','LT_TARGET_FCD'],)
df_supply_pcba=pd.read_excel(fname_supply,sheet_name='PCBA')
df_supply_df=pd.read_excel(fname_supply,sheet_name='DF')
df_ct2r=pd.read_excel(fname_ct2r)
df_bom=pd.read_excel(fname_bom)

In [4]:
# (different from PCBA GLO allocation methods)combine related supply data to df_supply:DF OH, scr,in transit, df components supply, based on availability date (ETA)
# and other necessary data processing
df_supply_pcba.set_index('PN',inplace=True)
df_supply_df.set_index('PN',inplace=True)
df_ct2r.set_index('PN',inplace=True)

col1=df_supply_pcba.columns
df_supply_pcba.columns=[x.date() for x in pd.to_datetime(col1)]

col2=df_supply_df.columns
df_supply_df.columns=[x.date() for x in pd.to_datetime(col2)]

df_3a4=df_3a4[df_3a4.BUSINESS_UNIT=='PABU']

df_3a4.CURRENT_FCD_NBD_DATE = pd.to_datetime(df_3a4.CURRENT_FCD_NBD_DATE)
df_3a4.ORIGINAL_FCD_NBD_DATE=pd.to_datetime(df_3a4.ORIGINAL_FCD_NBD_DATE)
df_3a4.TARGET_SSD=pd.to_datetime(df_3a4.TARGET_SSD)
df_3a4.LT_TARGET_FCD=pd.to_datetime(df_3a4.LT_TARGET_FCD)
df_3a4.CURRENT_FCD_NBD_DATE = df_3a4.CURRENT_FCD_NBD_DATE.map(lambda x: x.date() if x is not np.nan else np.nan)
df_3a4.ORIGINAL_FCD_NBD_DATE=df_3a4.ORIGINAL_FCD_NBD_DATE.map(lambda x: x.date() if x is not np.nan else np.nan)
df_3a4.TARGET_SSD=df_3a4.TARGET_SSD.map(lambda x: x.date() if x is not np.nan else np.nan)
df_3a4.LT_TARGET_FCD=df_3a4.LT_TARGET_FCD.map(lambda x: x.date() if x is not np.nan else np.nan)


In [106]:
# merge BOM into 3A4
def update_bom_to_3a4(df_3a4,df_bom):
    df_3a4=pd.merge(df_3a4,df_bom,left_on='PRODUCT_ID',right_on='PID',how='outer')
    
    df_3a4.loc[:,'PN']=np.where(df_3a4.TAN.notnull() & df_3a4.PN.isnull(),
                               df_3a4.TAN,
                               df_3a4.PN)
    
    missing_bom_pid=df_3a4[df_3a4.TAN.notnull() & df_3a4.PN.isnull()].PRODUCT_ID.unique()
    df_missing_bom_pid=pd.DataFrame({'Missing BOM PID':missing_bom_pid})
    
    return df_3a4,df_missing_bom_pid


In [111]:
df_3a4.columns

Index(['SO_SS', 'PO_NUMBER', 'OPTION_NUMBER', 'ORGANIZATION_CODE',
       'BUSINESS_UNIT', 'PRODUCT_FAMILY', 'PRODUCT_ID', 'TAN',
       'PID_DESCRIPTION', 'MFG_HOLD', 'ADDRESSABLE_FLAG', 'TIED_SHIP_SET',
       'SECONDARY_PRIORITY', 'GLOBAL_RANK', 'BUP_RANK', 'FINAL_ACTION_SUMMARY',
       'CONSOLIDATED_ALLOCATION_PRIORITY', 'ORDER_HOLDS', 'ORDERED_QUANTITY',
       'PACKOUT_QUANTITY', 'C_STAGED_QTY', 'C_UNSTAGED_QTY',
       'BUILD_COMPLETE_DATE', 'ASN_CREATION_DATE', 'ASN_PROCESSED_STATUS',
       'ORDERED_DATE', 'BOOKED_DATE', 'LINE_CREATION_DATE', 'LT_TARGET_FCD',
       'TARGET_SSD', 'CURRENT_FCD_NBD_DATE', 'ORIGINAL_FCD_NBD_DATE',
       'CUSTOMER_REQUEST_DATE', 'CUSTOMER_REQUESTED_SHIP_DATE',
       'ORIGINAL_PROMISE_DATE', 'TARGET_PROMISE_DATE', 'CURRENT_PROMISE_DATE',
       'EAD', 'ORDER_AGING_BUCKET', 'C_UNSTAGED_DOLLARS', 'SOL_REVENUE',
       'C_STAGED_DOLLARS', 'TAA', 'STD_VS_TWO_TIER', 'CARTON_LOCATION',
       'CONSOLIDATED_FLAG', 'REVENUE_NON_REVENUE', 'END_CUSTOMER_C

In [6]:
# 根据df_ct2r和df_supply_df判断假定df supply OK的日期 
def df_pn_ct2r_date_judgement(df_ct2r,df_supply_df):
    pn_ct2r_lastsupply_max={}
    max_date=df_supply_df.columns[-1]
    
    # 先把ct2r对应的PN及date加入字典 - 仅在ct2r date不大于supply df 中的最大日期时才添加（超出则认为没有supply）
    for row in df_ct2r.itertuples(index=True):
        #print(row)
        ct2r_date=pd.Timestamp.now().date()+pd.Timedelta(row.CT2R,'d')
        if ct2r_date<=max_date:
            pn_ct2r_lastsupply_max[row.Index]=ct2r_date
    
    # iter df_supply_df,从后向前检查每一个PN的supply,如果>0的值对应的date>pn_ct2r_lastsupply_max,则将日期改成对应的date;否则不变
    df_with_ct2r_to_fix=df_supply_df.loc[pn_ct2r_lastsupply_max.keys()]  #只针对特定的PN
    date_col=df_with_ct2r_to_fix.columns
    
    for row in df_with_ct2r_to_fix.itertuples(index=True):
        row_data=row[::-1][:-1]   #将每一行数据倒置检查
        #print(row[::-1])
        supply_index = next((-row_data.index(x) for x in row_data if x>0), '0_supply')  # 找到>0的第一个值，存储相应的负值（倒数位置）
        if supply_index!='0_supply': #有>0的值，取大的date并加1
            pn_ct2r_lastsupply_max[row.Index]=max(date_col[supply_index-1],pn_ct2r_lastsupply_max[row.Index])+pd.Timedelta(1,'d')
        else:                        #没有>0的值，保留ct2r date并加1
            pn_ct2r_lastsupply_max[row.Index]=pn_ct2r_lastsupply_max[row.Index]+pd.Timedelta(1,'d')
        
    return pn_ct2r_lastsupply_max


In [7]:
pn_ct2r_lastsupply_max=df_pn_ct2r_date_judgement(df_ct2r,df_supply_df)

/Users/wangken/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  


In [8]:
# 修改df_supply_df：对应日期赋值10000 --- 此模块运行时间太长，需要优化！！！！！！！！！！！！！！！！！
def update_supply_for_df_w_ct2r(df_supply_df,pn_ct2r_lastsupply_max):
    for pn,date in pn_ct2r_lastsupply_max.items():
        df_supply_df.loc[pn,date]=9999
    
    #按日期排序
    col=df_supply_df.columns
    df_supply_df=df_supply_df[sorted(col)]
    
    return df_supply_df

In [9]:
df_supply_df=update_supply_for_df_w_ct2r(df_supply_df,pn_ct2r_lastsupply_max)

In [33]:
#合并pcba和df supply
df_supply=pd.concat([df_supply_pcba,df_supply_df],sort=True)


In [36]:
# change df_supply and df_bom PN into versionless. qty added up to versionless PN in df_supply


In [116]:
regex=re.compile(r'\d{2,3}-\d{4,7}')
df_supply.index = df_supply.index.map(lambda x: regex.search(x).group())
df_supply.sort_index(inplace=True)

df_supply.reset_index(inplace=True)
dup_pn=df_supply[df_supply.duplicated('PN')].PN.unique()
df_sum=pd.DataFrame(columns=df_supply.columns)
df_sum.set_index('PN',inplace=True)
df_supply.set_index('PN',inplace=True)

for pn in dup_pn:
    #print(df_supply[df_supply.PN==pn].sum(axis=1).sum())
    df_sum.loc[pn,:]=df_supply.loc[pn,:].sum(axis=0)

df_supply.drop(dup_pn,axis=0,inplace=True)
df_supply=pd.concat([df_supply,df_sum])


In [ ]:
#TDO: add BOM to 3A4

In [14]:
# set up the default tan that have to be considered to calculate the PO material ready date - 临时设置为scr和3a4包含的tan
must_have_tan_list=df_supply.index.tolist()
must_have_tan_list=np.intersect1d(must_have_tan_list,df_3a4.TAN.unique().tolist())


NameError: name 'df_3a4' is not defined

In [ ]:
# sort and rank the orders (订单排序及优先可以有多种不同的排序 - 以下是一种临时性的方法)
df_3a4=df_3a4.sort_values(by=['SECONDARY_PRIORITY','C_UNSTAGED_DOLLARS','CURRENT_FCD_NBD_DATE','ORIGINAL_FCD_NBD_DATE', 'LT_TARGET_FCD'],
              ascending=True)

In [7]:
# define func for creating supply dic by tan&date
def created_supply_dict_per_df_supply(df_supply):
    """
    create supply dict based on df_supply
    supply_dic_tan={'800-42373-01':[{'2/10':25},{'2/12':4},{'2/15':10},{'2/22':20},{'3/1':10},{'3/5':15}],
               '800-42925-01':[{'2/12':4},{'2/13':3},{'2/15':12},{'2/23':25},{'3/1':8},{'3/6':10}]}
    """
    supply_dic_tan={}
    for tan in df_scr.index:
        date_qty_list=[]
        for date in df_scr.columns:
            qty=df_scr.loc[tan,date.date()]
            if qty>0:
                date_qty={date:df_scr.loc[tan,date.date()]}
                date_qty_list.append(date_qty)

        supply_dic_tan[tan]=date_qty_list
    
    return supply_dic_tan

In [8]:
# 根据tan list和df_3a4生成blg_dic_tan
def create_blg_dict_per_sorted_3a4_and_selected_tan(df_3a4,tan):
    """
    create backlog dict for selected tan list from the sorted 3a4 df (considered order prioity and 
    rank - a "production dequence")
    tan_list: from a predefined tan list, or from tans included in the supply df - depends on how we define it
    """
    blg_dic_tan={}
    for pn in tan:
        dfm=df_3a4[df_3a4.TAN==pn]
        org_qty_po=[]
        for org,qty,po,option,t_fcd,c_fcd in zip(dfm.ORGANIZATION_CODE,dfm.C_UNSTAGED_QTY,dfm.PO_NUMBER,dfm.OPTION_NUMBER,dfm.LT_TARGET_FCD,dfm.CURRENT_FCD_NBD_DATE):
            if qty>0:
                org_qty_po.append({org:(qty,(po,option,t_fcd,c_fcd))})

        blg_dic_tan[pn]=org_qty_po
    
    return blg_dic_tan

In [9]:
# For testing
"""
supply_dic_tan={}
supply_dic_tan['68-4680-01']=x

blg_dic_tan={}
blg_dic_tan['68-4680-01']=y
"""

"\nsupply_dic_tan={}\nsupply_dic_tan['68-4680-01']=x\n\nblg_dic_tan={}\nblg_dic_tan['68-4680-01']=y\n"

In [32]:
@jit
def allocate_supply_to_backlog(supply_dic_tan,blg_dic_tan):
    """
    allocate supply to each PO_OPTION by TAN based on supply dict and backlog dict
    supply dict is aranged in date order; backlog dict is aranged based on priority to fulfill
    
    examples: 
        blg_dic_tan={'800-42373-01': [{'FJZ': (5, ('110077267-1',0))},{'FJZ': (23, ('110011089-4',0))},...]}
        supply_dic_tan={'800-42373-01':[{'2/10':25},{'2/12':4},{'2/15':10},{'2/22':20},{'3/1':10},{'3/5':15}],
                             '800-42925-01':[{'2/12':4},{'2/13':3},{'2/15':12},{'2/23':25},{'3/1':8},{'3/6':10}]}
    return: blg_with_allocation contains {(po,option):([(supply date,qty)],last supply date)}
    """
    blg_with_allocation={}
    
    for tan in blg_dic_tan.keys():
        blg_list_tan=blg_dic_tan[tan]  #每一个tan对应的blg list

        if tan in supply_dic_tan.keys(): # 这一步总是成立，因为blg_dic_tan是根据supply 中的tan生成的 --- 可以考虑去除此判断
            supply_list_tan=supply_dic_tan[tan]
            #print(supply_list_tan)
            
            # 按顺序对每一个po_option进行数量分配
            for po_option in blg_list_tan:
                po_option_supply_allocation=[]
                last_supply_date=None
                #allocated_supply=[] #已经分配完的supply
                    
                po_option_qty=list(po_option.values())[0][0]
                po_number=list(po_option.values())[0][1][0]
                option_number=list(po_option.values())[0][1][1]
                target_fcd=list(po_option.values())[0][1][2]
                current_fcd=list(po_option.values())[0][1][3]
                
                target_fcd_short_qty=po_option_qty
                current_fcd_short_qty=po_option_qty
                
                # 按顺序将supply分给此po_option
                for date_qty in supply_list_tan:
                    #print(date_qty)
                    #print(type(date_qty))
                    supply_date=list(date_qty.keys())[0]
                    supply_qty=list(date_qty.values())[0]
                
                    if po_option_qty<supply_qty: #po_option数量小于supply数量：po_option被全额满足；supply数量被减掉；已分配的supply被记录 （后面跳转到下一个po）
                        po_option_supply_allocation.append((supply_date.strftime('%Y-%m-%d'),po_option_qty))
                        
                        #更新supply_list_tan
                        new_supply_qty=supply_qty-po_option_qty
                        ind=supply_list_tan.index(date_qty)
                        supply_list_tan[ind]={supply_date:new_supply_qty}
                        
                        po_option_qty=0
                        last_supply_date=supply_date.date()
                        
                        # 计算target_fcd_short_qty和current_fcd_short_qty
                        if supply_date<=target_fcd:
                            target_fcd_short_qty=0
                            
                        if supply_date<=current_fcd:
                            current_fcd_short_qty=0
                        
                        break
                    elif po_option_qty==supply_qty: #po_option数量等于supply数量：po_optio被全额满足；已分配的supply被记录；跳出本次po_option循环(进到下一个supply循环)
                        po_option_supply_allocation.append((supply_date.strftime('%Y-%m-%d'),po_option_qty))
                        
                        #更新supply_list_tan
                        new_supply_qty=0
                        ind=supply_list_tan.index(date_qty)
                        supply_list_tan[ind]={supply_date:new_supply_qty}
                        
                        po_option_qty=0
                        last_supply_date=supply_date.date()
                        
                        # 计算target_fcd_short_qty和current_fcd_short_qty
                        if supply_date<=target_fcd:
                            target_fcd_short_qty=0
                            
                        if supply_date<=current_fcd:
                            current_fcd_short_qty=0
                            
                        break
                    else: #po_option数量大于supply数量：po_option被部分满足（=supply qty）；po_option数量被改小；进到下一个supply循环(同一个po)
                        if supply_qty>0:
                            po_option_supply_allocation.append((supply_date.strftime('%Y-%m-%d'),supply_qty))
                            
                            #更新supply_list_tan
                            new_supply_qty=0
                            ind=supply_list_tan.index(date_qty)
                            supply_list_tan[ind]={supply_date:new_supply_qty}
                        
                            # 计算target_fcd_short_qty和current_fcd_short_qty
                            if supply_date<=target_fcd:
                                target_fcd_short_qty=target_fcd_short_qty-supply_qty

                            if supply_date<=current_fcd:
                                current_fcd_short_qty=current_fcd_short_qty-supply_qty
                            
                            #更新订单数量
                            po_option_qty=po_option_qty-supply_qty
                        
                        
                #完成一个po_option的分配，结果加入blg_with_allocation中
                uncovered_qty=po_option_qty
                blg_with_allocation[po_number+'-'+str(option_number)]=(po_option_supply_allocation,last_supply_date, uncovered_qty,target_fcd_short_qty,current_fcd_short_qty)
                        
             
    return blg_with_allocation



In [11]:
# calculate the PO supply ready date
def calculate_po_supply_ready_date(df_3a4,must_have_tan_list):
    dfx=df_3a4[df_3a4.TAN.isin(must_have_tan_list)].copy()
    
    dfx.sort_values(by=['PO_NUMBER','tan_supply_ready_date'],ascending=True,inplace=True)
    po_supply_date=[(x,y) for x, y in zip(dfx.PO_NUMBER,dfx.tan_supply_ready_date)]
    
    po_supply_ready_date={}
    po=po_supply_date[0][0]
    supply_date=po_supply_date[0][1]
    
    po_supply_ready_date[po]=supply_date  #第一个值
    
    for po_date in po_supply_date[1:]:
        if po_date[0]!=po or po_date[1]!=supply_date:
            po_supply_ready_date[po_date[0]]=po_date[1]
        po=po_date[0]
        supply_date=po_date[1]
    
    return po_supply_ready_date


In [12]:
def calculate_po_ctb(df_3a4):
    """
    Add 2 days FLT to get PO CTB based on PO supply available date
    Simplified approach to take space, need to be updated
    """
    df_3a4.loc[:,'po_ctb']=np.where(df_3a4.po_supply_ready_date.notnull(),
                                   df_3a4.po_supply_ready_date+pd.Timedelta(2,'d'),
                                   None)
    
    return df_3a4
    

In [13]:
def calculate_ss_ctb(df_3a4):
    """
    Calculate SS CTB based on PO CTB date - the latest one----- having bug to be fixed!!!!!!
    """
    
    ss=df_3a4[df_3a4.po_ctb.notnull()].SO_SS.unique()
    dfx=df_3a4[df_3a4.SO_SS.isin(ss)].copy()
    
    dfx.sort_values(by=['SO_SS','po_ctb'],ascending=True,inplace=True)
    po_ctb_date=[(x,y) for x, y in zip(dfx.SO_SS,dfx.po_ctb)]
    
    ss_ctb_date={}
    ss=po_ctb_date[0][0]
    ctb_date=po_ctb_date[0]
    
    ss_ctb_date[ss]=ctb_date  #第一个值
    
    for ss_date in po_ctb_date[1:]:
        if ss_date[0]!=ss or ss_date[1]!=ctb_date:
            ss_ctb_date[ss_date[0]]=ss_date[1]
        ss=ss_date[0]
        ctb_date=ss_date[1]
    
    return ss_ctb_date
    

In [14]:
def write_excel_file(fname, data_to_write):
    '''
    Write the df into excel files as different sheets
    :param fname: fname of the output excel
    :param data_to_write: a dict that contains {sheet_name:df}
    :return: None
    '''
    # engine='xlsxwriter' is used to avoid illegal character which lead to failure of saving the file
    writer = pd.ExcelWriter(fname,engine='xlsxwriter')

    for sheet_name, df in data_to_write.items():
        df.to_excel(writer, sheet_name=sheet_name)

    writer.save()
    

In [15]:
#  生成supply_dic_tan
supply_dic_tan=created_supply_dict_per_df_supply(df_scr)


In [16]:
# create backlog dict for Tan require allocation
tan=list(supply_dic_tan.keys())
blg_dic_tan=create_blg_dict_per_sorted_3a4_and_selected_tan(df_3a4,tan)


In [17]:
# 生成订单分配supply的结果 
blg_with_allocation=allocate_supply_to_backlog(supply_dic_tan,blg_dic_tan)

In [18]:
# 将分配结果加入到df_3a4中
df_3a4.OPTION_NUMBER=df_3a4.OPTION_NUMBER.astype(str)
df_3a4.loc[:,'po_option']=df_3a4.PO_NUMBER+'-'+df_3a4.OPTION_NUMBER

df_3a4.loc[:,'supply_allocation']=df_3a4.po_option.map(lambda x: blg_with_allocation[x][0] if x in blg_with_allocation.keys() else None)
df_3a4.loc[:,'tan_supply_ready_date']=df_3a4.po_option.map(lambda x: blg_with_allocation[x][1] if x in blg_with_allocation.keys() else None)
df_3a4.loc[:,'tan_qty_wo_supply']=df_3a4.po_option.map(lambda x: blg_with_allocation[x][2] if x in blg_with_allocation.keys() else None)
df_3a4.loc[:,'target_fcd_short_qty']=df_3a4.po_option.map(lambda x: blg_with_allocation[x][3] if x in blg_with_allocation.keys() else None)
df_3a4.loc[:,'current_fcd_short_qty']=df_3a4.po_option.map(lambda x: blg_with_allocation[x][4] if x in blg_with_allocation.keys() else None)



In [19]:
# calculate the PO supply ready date
po_supply_date=calculate_po_supply_ready_date(df_3a4,must_have_tan_list)

# 将po supply ready date加入3a4中
df_3a4.loc[:,'po_supply_ready_date']=df_3a4.PO_NUMBER.map(lambda x: po_supply_date[x] if x in po_supply_date.keys() else None)


In [20]:
# TODO: allocate capacity to PO


In [21]:
# TODO: calculate PO CTB date (should consider FLT) -- using on build projection - need to update consider capacity allocation result
df_3a4=calculate_po_ctb(df_3a4)


In [22]:
# TODO: calculate SS CTB date -- used on pull in/decommit calculation
ss_ctb_date=calculate_ss_ctb(df_3a4)
df_3a4.loc[:,'ss_ctb']=df_3a4.SO_SS.map(lambda x: ss_ctb_date[x] if x in ss_ctb_date.keys() else None)


In [23]:
# Calculate PO status
df_3a4.loc[:,'ss_updated_status']=np.where((df_3a4.CURRENT_FCD_NBD_DATE.notnull()&df_3a4.ss_ctb.notnull()),
                                           np.where(df_3a4.CURRENT_FCD_NBD_DATE>pd.to_datetime(df_3a4.ss_ctb),
                                                    'Pull in opportunity',
                                                    np.where(df_3a4.CURRENT_FCD_NBD_DATE<pd.to_datetime(df_3a4.ss_ctb),
                                                            'Decommit risk',
                                                            'On schedule')),
                                           None)
                                           


In [24]:
# make summaries - samples. need to modify and correct or improve. should consider to use PO_REV for the summaries
df_ctb=df_3a4[df_3a4.TAN.isin(df_scr.index)].pivot_table(index='BUSINESS_UNIT',columns='po_supply_ready_date',values='C_UNSTAGED_DOLLARS',aggfunc=sum)
shortage_target_fcd=df_3a4[df_3a4.target_fcd_short_qty.notnull()].pivot_table(index='TAN',columns='LT_TARGET_FCD',values='C_UNSTAGED_DOLLARS',aggfunc=sum)
ss_status=df_3a4[df_3a4.TAN.isin(df_scr.index)].pivot_table(index='ss_updated_status',columns='CURRENT_FCD_NBD_DATE',values='C_UNSTAGED_DOLLARS',aggfunc=sum)   

col=['ORGANIZATION_CODE','BUSINESS_UNIT', 'PRODUCT_FAMILY', 'PRODUCT_ID', 'SO_SS','PO_NUMBER', 'OPTION_NUMBER','TAN','ORDERED_QUANTITY','C_UNSTAGED_QTY','C_UNSTAGED_DOLLARS',
      'CURRENT_FCD_NBD_DATE','LT_TARGET_FCD','tan_qty_wo_supply','tan_supply_ready_date', 'po_supply_ready_date', 'po_ctb','ss_ctb','ss_updated_status','target_fcd_short_qty','current_fcd_short_qty','supply_allocation']
# unfinished
df_3a4=df_3a4[col]

    

In [26]:
# write to excel
data_to_write={'3a4':df_3a4,
              'ctb':df_ctb,
              'ss_status':ss_status,
              'shortage':shortage_target_fcd}
fname='ctb output.xlsx'
write_excel_file(fname, data_to_write)


In [ ]:
df_3a4.columns

In [130]:
col=['PO_NUMBER','OPTION_NUMBER','PRODUCT_ID','TAN','ORDERED_QUANTITY']

In [131]:
df_3a4[df_3a4.PO_NUMBER=='109969501-48'][col]

,PO_NUMBER,OPTION_NUMBER,PRODUCT_ID,TAN,ORDERED_QUANTITY
1490,109969501-48,0,ASR-903,68-3992-01,20
1491,109969501-48,1,ASR900 FAN TRAY,NaN,20
1492,109969501-48,2,A903-FAN,800-35305-02,20
1493,109969501-48,3,ASR900 RSP SLOT 0,NaN,20
1494,109969501-48,4,A900-RSP2A-128,68-4680-01,20
1495,109969501-48,5,ASR900 RSP SLOT 1,NaN,20
1496,109969501-48,6,A900-RSP2A-128,68-4680-01,20
1497,109969501-48,7,INTF MOD SLOT 0,NaN,20
1498,109969501-48,8,A900-IMA8T1Z,68-4754-01,20
1499,109969501-48,9,INTF MOD SLOT 1,NaN,20
